In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from io import StringIO
from scipy.stats import f

# 实践中的统计
## Monsanto公司
Monsanto公司已经成为美国最大的化学公司之一，生产1000多种产品，从工业化学制品到用于铺设现代运动场所需要的人工合成地面，应有尽有  
Monsanto公司最终的成功取决于他们发现了肉鸡饲料的最优配方，使得肉鸡的饲养成本与猪、牛等其他家畜的饲养成本相比，保持在更低的水平上  
Monsanto公司运用回归分析方法模拟了肉鸡的体重y与饲料中加入的蛋氨酸数量x之间的关系。最初建立的估计的简单线性回归方程如下：  
$$\hat{y}=0.21+0.42x$$
这一估计的回归方程经检验在统计上是显著的，但是残差分析显示，曲线关系也许是描述肉鸡体重与饲料中加入蛋氨酸数量之间关系的一个更适宜的模型 5  
Monsanto公司经过进一步的研究发现，虽然少了的蛋氨酸可以提高肉鸡体重，但是当其含量达到某一水平后，继续增加蛋氨酸的含量对肉鸡体重增长的作用就变得微乎其微了  
下面估计的多元回归方程用于模拟肉鸡体重与饲料中加入的蛋氨酸数量之间的曲线关系  
$$\hat{y}=-1.89+1.32x-0.506x^2$$
利用这一回归结果，Monsanto公司能够确定在肉鸡饲料中需要添加蛋氨酸的最优数量  
在本章中，我们将通过对诸如Monsanto公司建模的例子，扩展回归分析的讨论至非线性模型 5  
建立模型是一个复杂的过程，经过这一过程，我们就能得到描述应变量与一个或一个以上自变量之间关系的估计的回归方程  
建模过程主要有两方面的问题：一是找到一个合适的描述变量之间关系的函数形式，二是选择模型应包含的自变量 5  
# 16.1 一般线性模型
假设我们采集了一个应变量y和k个自变量$x_1,x_2,\dots,x_k$的观测数据。作为建立自变量之间更复杂关系的总体框架，我们介绍含有p个自变量的**一般线性模型**的概念  
<hr />

**一般线性模型**（16-1）  
$$y=\beta_0+\beta_1z_1+\beta_2z_2+\dots+\beta_pz_p+\varepsilon$$
<hr />

在式（16-1）中，每一个自变量$z_j$都是$x_1,x_2,\dots,x_k$的函数  
最简单的情形是我们仅仅对一个变量x的函数，并且希望利用直线关系去估计y。在这种情形下，$z_1=x_1$，式（16-1）变为（16-2） 5  
$$y=\beta_0+\beta_1x_1+\varepsilon$$
在统计建模文献中，这个模型被称为具有一个预测变量的简单一阶模型  
## 16.1.1 模拟曲线关系
我们能用式（16-1）模拟形式更复杂的关系。Reynolds公司的管理人员希望对公司销售人员工作年限的长短和电子实验室天平的销售数量之间的关系进行调研  
表16-1给出了15名随机抽选的销售人员近期天平的销售数量和每一名销售人员被公司雇佣的月数  

In [ ]:
'''
5
'''
rey_data=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/Reynolds.csv')
rey_data.tail()

图16-1给出了这些数据的散点图  

In [ ]:
rey_data.plot.scatter('Months','Sales')
plt.ylim(0,400)
plt.xlim(0,120)
plt.show()

散点图表明，在被公司雇佣时间的长短和销售数量之间，可能存在一个曲线关系 5  
在考虑如何为Reynolds公司建立一个曲线关系之前，让我们首先考虑图16-2中给出的与简单一阶模型对应的Minitab输出，估计的回归方程是  

In [ ]:
'''
python	patsy	formula	~
'''
rey_formula_sap='Sales ~ Months'
rey_mod_sap=smf.ols(rey_formula_sap,rey_data).fit()

print('Sales={:.0f}+{:.2f}Months'.format(rey_mod_sap.params[0],rey_mod_sap.params[1]))

In [ ]:
rey_mod_sap.summary()

图16-3是对于的标准化残差图 5  

In [ ]:
'''
5
'''
rey_inf_sap=rey_mod_sap.get_influence()
rey_inf_sut=rey_inf_sap.summary_table()
rey_str_lv1=StringIO(rey_inf_sut.as_csv())
rey_sactter_lv1=pd.read_csv(rey_str_lv1,header=0,skiprows=[1],usecols=[2,4])
rey_sactter_lv1.columns=['hy','stdRes']
rey_sactter_lv1.plot.scatter('hy','stdRes')
plt.plot(np.array([100,400]),np.array([0,0]),linestyle='--')
plt.xlim(100,400)
plt.show()

尽管计算机输出表名：这个线性关系是显著的，并且线性关系解释了销售数量中的大部分变异性，然而标准化残差图启发我们，仍然需要一个曲线关系 5  
为了说明这是一个曲线关系，我们令式（16-1）中的$z_1=x_1,z_2=x_1^2$，于是得到模型（16-3）  
$$y=\beta_0+\beta_1x_1+\beta_2x_1^2+\varepsilon$$
这个模型被称为具有一个预测变量的二阶模型  
为了建立与这个二阶模型相对应的估计的回归方程，我们使用的统计软件包不但需要表16-1中的原始数据，而且也需要增加第二个自变量所对应的数据，即销售人员被公司雇佣月数的平方  
在图16-4中，我们给出了对应二阶模型的Minitab输出，估计的回归方程是 5  

In [ ]:
'''
python	patsy	formula	function()
'''
rey_formula_lv2='Sales ~ Months + np.power(Months,2)'
rey_mod_lv2=smf.ols(rey_formula_lv2,rey_data).fit()
print('Sales={:.1f}+{:.2f}Months+{:.4f}MonthsSq'.format(rey_mod_lv2.params[0],
                                                        rey_mod_lv2.params[1],
                                                       rey_mod_lv2.params[2]))

式中，MonthsSq代表销售人员被公司雇佣月数的平方  

In [ ]:
rey_mod_lv2.summary()

图16-5是对应的标准化残差图 5  

In [ ]:
'''
5
'''
def df_std_resid(mod):
    '''
    统计常用代码	简单线性回归	残差分析：证实模型的假定	标准化残差
    5
    '''
    inf=mod.get_influence()
    inf_sut=inf.summary_table()
    str_io=StringIO(inf_sut.as_csv())
    sactter=pd.read_csv(str_io,header=0,skiprows=[1],usecols=[2,4])
    sactter.columns=['yhat','stdRes']
    return sactter

rey_sactter_lv2=df_std_resid(rey_mod_lv2)
rey_sactter_lv2.plot.scatter('yhat','stdRes',figsize=(7,7))
plt.plot(np.array([0,350]),np.array([0,0]),linestyle='--')
plt.xlim(50,350)
plt.ylim(-3,2)
plt.show()

这张图表明：前面图16-3中的曲线模式的图形已经被消除  
在$\alpha=0.05$的显著性水平下，计算机输出表明，模型在总体上是显著的;我们还注意到，对应于自变量MonthsSq的t检验的p-值小于0.05，因此我们的结论是，在含有自变量Months的模型中，增加的自变量MonthsSq是显著的  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.rsquared_adj
'''
print('由于R-sq={:.1%},由这个估计的回归方程给出的拟合我们应该是满意的'.format(rey_mod_lv2.rsquared_adj))

在多元回归分析中，线性这个词在术语“一般线性模型”中指的仅仅是这样一个事实：$\beta_0,\beta_1,\dots,\beta_p$全是一次幂，这并不意味着y和这些$x_i$之间存在着线性关系  
## 16.1.2 交互作用
如果原始数据集由应变量y和两个应变量$x_1,x_2$的观测值组成，在一般线性模型式（16-1）中，设$z_1=x_1,z_2=x_2,z_3=x_1^2,z_4=x_2^2和z_5=x_1x_2$，我们就能建立一个含有两个预测变量的二阶模型 5  
我们得到的模型是（16-4）  
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_1^2+\beta_4x_2^2+\beta_5x_1x_2+\varepsilon$$
在这个二阶模型中，为了说明两个变量共同作用产生的潜在影响，我们增加了一个变量$z_5=x_1x_2$。这种类型的影响称为**交互作用**  
Tyler认为，对销售量起最大影响的两个因素是单位销售价格和广告费用。为了研究这两个变量对销售量的影响，在24家做实验的商店中，与价格为2美元、2.5美元和3美元相对应的广告费用分别为50000美元和100000美元  
我们观测到的销售数量（单位：1000瓶）记录在表16-2中 5  

In [ ]:
tyl=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/Tyler.csv')
tyl.head()

表16-3是这些数据的汇总  

In [ ]:
'''
5
'''
tyl.columns=['Price','Advertising','Sales']
tyl.pivot_table(values='Sales',index='Advertising',columns='Price')

因此，当价格为2美元保持不变时，广告费用分别为50000美元和100000美元，这时候的平均销售数量之差是808-461=347  
当产品价格为2.5美元时，平均销售数量之差是646-364=282。最后，当产品的价格为3美元时，平均销售数量之差依赖于产品的销售价格  
显然，广告费用分别为50000美元和100000美元时，平均销售数量之差依赖于产品的销售价格  
换句话说，当销售价格较高时，增加广告费用的影响将要减少。上述观测结果提供了销售价格和广告费用这两个变量之间交互作用的证据  
为了给出交互作用的另一个观点，图16-6表示了6种不同的销售价格与广告费用组合的平均销售数量  
![16-6](../syn_pic/statistics_for_business_economics/16-6.png)
<center>图16-6 平均销售数量是销售价格和广告费用的函数 5</center>

换句话说，只有当我们考虑两个变量对响应变量的联合影响时，才能得出有意义的结论  
为了说明交互作用的影响，我们将利用下面的回归模型（16-5）  
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_3+\varepsilon$$
式中，y代表销售数量;$x_1$代表销售价格（美元）;$x_2$代表广告费用（1000美元）  
注意：式（16-5）反映出Tyler相信销售数量线性地依赖销售价格和广告费用，并且两个变量之间存在交互作用 5  
为了建立估计的回归方程，我们利用含有3个自变量的一般线性模型（16-6）  
$$y=\beta_0+\beta_1z_1+\beta_2z_2+\beta_3z_3+\varepsilon$$
式，$z_1$代表$x_1$;$z_2$代表$x_2$；$z_3$代表$x_1x_2$  
图16-7是对应于Tyler Personal 例子的交互作用模型的Minitab输出，得到的估计方程是  

In [ ]:
'''
5
python	patsy	formula	:
'''
tyl_formula='Sales ~ Price + Advertising + Price:Advertising'
tyl_mod=smf.ols(tyl_formula,tyl).fit()
print('Sales={:.0f}+{:.0f}Price+{:.1f}AdvExp{:.2f}PriceAdv'.format(tyl_mod.params[0],
                                                                   tyl_mod.params[1],
                                                                   tyl_mod.params[2],
                                                                   tyl_mod.params[3]))

式中，Sales代表销售数量;Price代表销售价格;AdvExp代表广告费用;PriceAdv代表交互作用项  

In [ ]:
tyl_mod.summary()

因为模型在总体上是显著的，对应于交互作用项PriceAdv的t检验的p-值是0.000,我们的结论是:在已知产品销售价格和广告费用的线性影响下，交互作用是显著的 5  
## 16.1.3 涉及应变量的变换
在说明如何利用一般线性模型模拟自变量和应变量之间各种可能存在的关系时，我们把注意力集中到包含一个或一个以上自变量的变换上  
为了解释说明时候应该对应变量进行变换，我们考虑表16-4中有关12辆汽车的英里/加仑额定值和重量的数据  

In [ ]:
mpg=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/MPG.csv')
mpg.head()

In [ ]:
mpg.columns=['Weight','Miles']

散点图16-8表明了在这两个变量之间存在一个负的线性关系 5  

In [ ]:
mpg.plot.scatter('Weight','Miles')
plt.ylim(0,35)
plt.xlim(2000,3800)
plt.show()

所以我们利用一个简单的一阶模型把这两个变量联系起来。图16-9是Minitab输出，得到估计的回归方程是  

In [ ]:
'''
5
'''
mpg_formula='Miles ~ Weight'
mpg_mod=smf.ols(mpg_formula,mpg).fit()
print('MPG={:.1f}{:.4f}Weight'.format(mpg_mod.params[0],
                                     mpg_mod.params[1]))

式中，MPG代表英里/加仑额定值;Weight代表汽车的重量（磅）  

In [ ]:
mpg_mod.summary()

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.rsquared
'''
print('模型是显著的，并且数据拟合得非常好(R-sq={:.1%})'.format(mpg_mod.rsquared))

然而，我们注意到在图16-9中，第三个观测值被识别出有一个大的标准化残差  

In [ ]:
'''
5
python	statsmodels	Statistics stats	OLSInfluence.resid_studentized_internal
'''
mpg_inf=mpg_mod.get_influence()
mpg_inf.resid_studentized_internal>2

In [ ]:
'''
统计常用代码	简单线性回归	残差分析：证实模型的假定	标准化残差
'''
mpg_std_df=df_std_resid(mpg_mod)
mpg_std_df.columns

图16-10是对应于一阶模型的标准化残差图  

In [ ]:
'''
5
'''
mpg_std_df.plot.scatter('yhat', 'stdRes')
plt.ylim(-2,2.5)
plt.xlim(10,35)
plt.plot(np.array([10,35]),np.array([0,0]),linestyle='--')
plt.show()

如果关于误差项的假设全都成立，我们期望看到的应是一条水平的带状图形，然而我们观察到的图形看起来不像是这种形状  
实际上，残差的变异性看来随着$\hat{y}$值的增加而增加。当显著性检验的基本假设看起来没有得到满足时，我们不能证明：得到的有关估计的回归方程统计显著性的任何结论  
非常数方程问题通常能够得到修正，修正的方法是对应变量做一个不同的比例变换。例如，用应变量的对数来代替原来的应变量，这样做的效果是压缩了应变量的数值，从而达到减少非常数方差影响的目的  
我们用英里/加仑的自然对数作应变量，在输出中的标记为LogeMPG，得到的回归结果在图16-11中，图16-12是对应的标准化残差图  

In [ ]:
'''
5
python	numpy	Mathematical functions	np.log()
python	patsy	formula	function()
'''
mpg_formula_log='np.log(Miles) ~ Weight'
mpg_mod_log=smf.ols(mpg_formula_log,mpg).fit()
mpg_mod_log.summary()

In [ ]:
mpg_inf_log=mpg_mod_log.get_influence()
mpg_inf_log.resid_studentized_internal>2

In [ ]:
'''
5
'''
mpg_std_log=df_std_resid(mpg_mod_log)
mpg_std_log.plot.scatter('yhat', 'stdRes')
plt.ylim(-2,2)
plt.xlim(2.6,3.6)
plt.plot(np.array([2.6,3.6]),np.array([0,0]),linestyle='--')
plt.show()

我们注意一下图16-12中的标准化残差图，现在我们已经看不到楔形图了。此外，没有一个观测值被识别出有一个大的标准化残差  
因此，我们愿意推荐估计的回归方程是  

In [ ]:
print('LogeMPG={:.2f}{:.6f}Weight'.format(mpg_mod_log.params[0],
                                         mpg_mod_log.params[1]))

对于一辆重量为2500磅的汽车，为了估计的它的英里/加仑额定值，我们首先应求出英里/加仑额定值的自然对数的估计值  

In [ ]:
'''
5
python	numpy	Mathematical functions	np.exp()
'''
print('我们得到{:.1f}英里/加仑额定值'.format(np.exp(mpg_mod_log.params[0]+mpg_mod_log.params[1]*2500)))

修正非常数方程问题的另一个方法是用1/y作应变量来代替原来的应变量y。这种类型的变换叫做倒数变换  
在一般情况下，没有方法能决定，究竟是进行对数变换还是进行倒数变换效果更好，除非对两种变换都实际地试一试  
## 16.1.4 内线性的非线性模型
参数($\beta_0,\beta_1,\dots,\beta_p$)的幂次超过一次的模型称为非线性模型。指数模型与下面的回归方程有关（16-7）  
$$E(y)=\beta_0\beta_1^x$$
当应变量y随着x的增加，按一个不变的百分比，而不是一个固定的数量增加或减少时，适合应用这种模型 5  
作为一个例子，假设一种产品的销售收入y依赖于广告费用x，这个问题对应的指数模型如下所示  
$$E(y)=500\times(1.2)^x$$
$对于x=1，E(y)=500\times(1.2)^1=600;对于x=2，E(y)=500\times(1.2)^2=720;对于x=3，E(y)=500\times(1.2)^3=864;$  
注意，在这种情形下，E(y)不是按一个固定的数量增加，而是按一个不变的百分比增加;增长的百分比是20%  
我们通过对式（16-7）两边取对数，将这个非线性模型转换为一个线性模型（16-8） 5  
$$logE(y)=log\beta_0+xlog\beta_1$$
注意，如果我们设$\acute{y}=logE(y),\acute{\beta}_0=log\beta_0,\acute{\beta}_1=log\beta_1$，我们能将式（16-8）写成  
$$\acute{y}=\acute{\beta}_0+\acute{\beta}_1x$$
显然，我们能利用简单线性回归的公式求出$\acute{\beta}_0和\acute{\beta}_1$的估计量。用$\acute{b_0}$和$\acute{b_1}$表示估计量，得到下面估计的回归方程（16-9）  
$$\acute{\hat{y}}=\acute{b_0}+\acute{b_1}x$$
5  
$\acute{\hat{y}}$的反对数就是我们要求的y的预测值，或者是y的期望值  
许多非线性模型不能被转换为一个等价的线性模型。这种模型所必需的数学背景超出了本教科书的范围 5  
# 16.2 确定什么时候增加或者删除变量
在这一节我们将说明，如何而利用F检验来确定，将一个或一个以上的自变量增加到一个多元回归模型上是否适宜的问题  
这一检验的根据是：测定一个多元回归模型增加一个或一个以上的自变量所得到的误差平方和减少的数量   

In [ ]:
but=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch15/Butler.csv',index_col=0)
but

在第15章，我们用Butler运输公司的例子去说明多元回归分析的应用。只用运输车辆运输货物的行驶里程$x_1$作自变量，利用最小二乘法得到项目的估计的回归方程 

In [ ]:
'''
5
'''
but_formula='Time ~ Miles'
but_mod=smf.ols(but_formula,but).fit()
print('hat_y={:.2f}{:+.4f}x_1'.format(but_mod.params[0],
                                    but_mod.params[1]))

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.resid
'''
but_mod_sse=but_mod.resid.pow(2).sum()
print('在第15章我们已经给出了这个模型的误差平方和是SSE={:.3f}'.format(but_mod_sse))

当运输货物的次数$x_2$作为第二个自变量增加到模型上时，我们得到下面的估计的回归方程  

In [ ]:
'''
5
'''
but_formula_m='Time ~ Miles + Deliveries'
but_mod_m=smf.ols(but_formula_m,but).fit()
print('hat_y={:+.3f}{:+.4f}x_1{:+.3f}x_2'.format(but_mod_m.params[0],
                                    but_mod_m.params[1],
                                    but_mod_m.params[2]))

In [ ]:
but_mod_sse_m=but_mod_m.resid.pow(2).sum()
print('这个模型的误差平方和是SSE={:.3f}'.format(but_mod_sse_m))

显然，增加$x_2$导致SSE数量的减少。我们希望回答的问题是：增加变量$x_2$是否导致了SSE数量显著的减少？  
当模型中$x_1$是唯一的自变量时，我们用记号SSE($x_1$)表示模型的误差平方和;当模型中有两个自变量$x_1,x_2$时，我们用记号SSE($x_1,x_2$)表示模型的误差平方和，等等  
因此，在仅包含自变量$x_1$的模型中，将$x_2$增加到模型上，引起SSE减少的数量是 5  

In [ ]:
but_diff_sse=but_mod_sse-but_mod_sse_m
but_diff_sse

$$SSE(x_1)-SSE(x_1,x_2)=5.730$$
我们进行F检验去确定，这一数量上的减少是不是显著的  
F统计量的分子是用SSE减少的数量除以增加到原模型上的自变量的个数。这里，仅有一个自变量$x_2$增加到模型上，于是F统计量的分子是  
$$\frac{SSE(x_1)-SSE(x_1,x_2)}{1}=5.730$$
5  
得到的这个结果是模型每增加一个自变量，误差平方和SSE减少数量的度量。F统计量的分母是包括全部自变量的模型的均方误差。对于Butler运输公司的例子，对应的模型含有两个自变量$x_1$和$x_2$，于是p=2，并且有  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.nobs
python	statsmodels	Linear Regression	RegressionResults.df_model

'''
but_mod_mse_m=but_mod_sse_m/(but_mod_m.nobs-but_mod_m.df_model-1)
print('MSE=SSE(x1,x2)/(n-p-1)={:.4f}'.format(but_mod_mse_m))

下面的F统计量给出了将自变量$x_2$增加到模型上，在统计上是否显著的检验（16-10）  
$$F=\frac{\frac{SSE(x_1)-SSE(x_1,x_2)}{1}}{\frac{SSE(x_1,x_2)}{n-p-1}}$$
5  
这个F检验的分子的自由度等于增加到模型中的自变量的个数，分母的自由度等于n-p-1  
对于Butler运输公司的例子，我们得到  

In [ ]:
but_f=but_diff_sse/but_mod_mse_m
print('F={:.2f}'.format(but_f))

In [ ]:
'''
5
python	scipy	Statistical functions (scipy.stats)	f() f.sf()

'''
but_p=f.sf(17.44,1,but_mod_m.nobs-but_mod_m.df_model-1)
print('对于显著性水平alpha=0.05来说，由于p-值={:.3f}<0.05，所以我们拒绝原假设，增加自变量x2到模型上，将引起误差平方和显著的减少'.format(but_p))

## 16.2.1 一般情形
考虑以下含有q个自变量的多元回归模型，这里q < p(16-11)  
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\dots+\beta_qx_q+\varepsilon$$
如果增加自变量$x_{q+1},x_{q+2},\dots,x_p$到这个模型上，我们就得到一个含有p个自变量的多元回归模型（16-12） 
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\dots+\beta_qx_q+\beta_{q+1}x_{q+1}+\beta_{q+2}x_{q+2}+\dots+\beta_px_p+\varepsilon$$
为了检验增加的自变量$x_{q+1},x_{q+2},\dots,x_p$是否在统计上是显著的，我们提出的原假设和备择假设叙述如下 5  
$$H_0:\beta_{q+1}=\beta_{q+2}=\dots=\beta_{p}=0$$
$$H_a:参数\beta_{q+1},\beta_{q+2},\dots,\beta_{p}中至少有一个不等于零$$  
下面的F统计量给出了检验增加的自变量$x_{q+1},x_{q+2},\dots,x_p$在统计上是否显著的根据（16-13）  
$$F=\frac{\frac{SSE(x_1,x_2,\dots,x_q)-SSE(x_1,x_2,\dots,x_q,x_{q+1},\dots,x_p)}{p-q}}{\frac{SSE(x_1,x_2,\dots,x_q,x_{q+1},\dots,x_p)}{n-p-1}}$$
5  
然后，将计算出的F统计量的值与分子的自由度为p-q、分母的自由度为n-p-1的F分布表的上侧分位数$F_\alpha$进行比较。如果$F>F_\alpha$，我们拒绝$H_0$，并且可以得出结论：增加的这组自变量在统计上是显著的  
为了给出F统计量一个比较简单的表述，我们将自变量个数较少的模型称为简化模型，将自变量个数较多的模型称为完全模型  
如果我们用SSE（简化）表示简化模型的误差平方和，用SSE（完全）表示完全模型的误差平方和，我们能把式（16-13）写成（16-14）  
$$\frac{SSE(简化)-SSE(完全)}{增加的项数}$$
注意，式（16-14）中的分母”增加的项数“表示完全模型的自变量个数和简化模型的自变量个数之间的差 5  
我们用MSE（完全）表示完全模型的均方误差，于是我们就能将式（16-13）写成（16-15）  
$$F=\frac{\frac{SSE(简化)-SSE(完全)}{增加的项数}}{MSE(完全)}$$
为了说明这个F统计量的应用，假设我们有一个含有30个观测值的回归问题。第一个模型的自变量是$x_1,x_2,x_3$，它的误差平方和为150;第二个模型的自变量是$x_1,x_2,x_3,x_4,x_5$，它的误差平方和为100。增加两个自变量$x_4,x_5$到第一个模型上，会引起误差平方和显著的减少吗？   
首先，我们注意到：SST的自由度是30-1=29，完全模型的回归平方和的自由度是5（在完全模型中自变量的个数）。于是，完全模型的误差平方和的自由度是24，因此MSE(完全)=100/24=4.17。所以F统计量是  

In [ ]:
'''
5
'''
F=((150-100)/2)/(100/(30-5-1))
F

## 16.2.2 p-值的应用
我们还能利用p-值准则来确定，增加一个或一个以上的自变量到一个多元回归模型上是否适宜的问题  

In [ ]:
'''
5
'''
print('由于p-值={:.3f}<alpha=0.05，我拒绝原假设，增加的自变量x4和x5在统计上是显著的'.format(f.sf(F,2,30-5-1)))

## 注释
F统计量的计算也能根据回归平方和的差来完成。为了说明这种计算形式的F统计量，首先，我们注意到  
$$SSE(简化)=SST-SSR(简化)$$
$$SSE(完全)=SST-SSR(完全)$$
所以
$$SSE(简化)-SSE(完全)=SSR(完全)-SSE(简化)$$
于是  
$$F=\frac{\frac{SSR(完全)-SSR(简化)}{自变量增加的个数}}{MSE(完全)}$$
5  
# 16.3 大型问题的分析
在介绍多元回归分析时，我们广泛地应用了Butler运输公司地例子  
为了给出将要在下一节详细论述的变量选择过程，我们引入由8个自变量、25组观测值组成的一个数据集  
由25个销售区域组成了一个随机样本，得到的数据如表16-5所示;变量的定义在表16-6中给出  

In [ ]:
cra=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/Cravens.csv')
cra.tail()

<center>表16-6 Cravens数据的变量定义 5</center>

![tb16-6](../syn_pic/statistics_for_business_economics/tb16-6.png)
作为第一步，我们考虑每一对变量之间的样本相关系数。图16-13是利用Minitab的相关命令得到的相关矩阵  

In [ ]:
'''
python	pandas	dataframe	d.corr()

'''
cra_corr=cra.corr()
cra_corr

In [ ]:
'''
python	pandas	dataframe	d.gt()
'''
cra_corr.gt(0.7)

我们观察以下自变量之间的样本相关系数，我们看到Time和Accounts之间的样本相关系数是0.758；因此，如何我们用Accounts作为一个自变量，那么Time将不会对模型增加更多的解释能力  
多重共线性的经验检验法则：对于任意两个自变量，如果样本相关系数的绝对值大于0.7，那么多重共线性可能对估计的结果产生影响 5  
因此，如果可能的话，我们应该避免Time和Accounts这两个自变量同时出现在一个回归模型中  
Change和Rating之间的样本相关系数是0.549，它也是比较高的，所以我们有理由要对这两个自变量做进一步考虑  
让我们观察一下Sales和每一个自变量之间的样本相关系数，很快我们就能知道，哪些自变量它们自己就是很好的预测变量    
我们看到，Accounts是Sales的一个最好的预测变量，因为它们之间的样本相关系数最高0.754  

In [ ]:
'''
5
python	pandas	series	s.sort_values() ascending
'''
cra_corr_sales=cra_corr['Sales']
cra_corr_sales.sort_values(ascending=False)

In [ ]:
print('于是，Accounts能解释Sales中的{:.2%}的变异性'.format(np.power(cra_corr_sales['Accounts'],2)))

虽然存在潜在的多重共线性，我们还是考虑利用全部8个自变量，建立一个估计的回归方程。Minitab计算机软件包给出的计算结果如图16-14所示  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.summary()
'''
cra_fomula='Sales ~ Accounts + Time + Poten + AdvExp + Change + Share + Rating + Work'
cra_mod=smf.ols(cra_fomula,cra).fit()
cra_mod.summary()

In [ ]:
'''
5
python	statsmodels	Linear Regression	RegressionResults.rsquared_adj
'''
print('8个自变量的多元回归模型的修正判定系数是{:.1%}'.format(cra_mod.rsquared_adj))

然而，我们注意到，对于单个参数的t检验，在$\alpha=0.05$的显著性水平下，当所有其他变量的影响已知时，仅有Poten、AdvExp和Share的p-值是显著的  
因此，我们或许倾向于研究仅仅利用这3个自变量而得到的模型。在图16-15中，Minitab给出了利用这3个自变量得到的估计的回归方程的结果  

In [ ]:
'''
5
'''
cra_fomula_lv1='Sales ~ Poten + AdvExp + Share'
cra_mod_lv1=smf.ols(cra_fomula_lv1,cra).fit()
cra_mod_lv1.summary()

In [ ]:
print('我们看到，估计的回归方程的修正判定系数是{:.1%}'.format(cra_mod_lv1.rsquared_adj))

尽管它不如利用全部8个自变量得到的估计回归方程那样好，但是这个修正判定系数也是非常高的  
在可供使用的数据已知的，我们如何才能求得一个具有最佳效果的估计的回归方程呢？一个方法是计算所有可能的回归方程。也就是说，我们要建立8个单变量的估计的回归方程，28个双变量的估计的回归方程，等等  
现在，我们已经拥有一些更出色、更高效的计算机软件包，它能计算出所有可能的回归方程  
但是这样一来就要涉及大量的计算，并且要求模型设计者审查大量的计算机输出结果，显然大部分的输出结果都与不好的模型相联系 5  
统计学家们更喜欢用系统的方法从全部自变量中选择一部分自变量，用这一部分自变量就能得到最优的估计的回归方程 5  
# 16.4 变量选择方法
在这一节，我们将讨论4种**变量选择方法**:逐步回归、前向选择、后向消元和最佳子集回归  
在逐步回归、前向选择、后向消元方法的每一步中，是增加一个自变量到回归模型上，还是从回归模型中删除一个自变量，选择自变量的准则是我们在16.2节中介绍的F统计量  
为了检验增加或者删除的自变量$x_2$是否在统计上是显著的，我们提出的原假设和备择假设表示成如下形式  
$$H_0:\beta_2=0$$
$$H_a:\beta_2\ne0$$
在16.2节，我们给出了下面的F统计量： 5  
$$F=\frac{\frac{SSE(x_1)-SSE(x_1,x_2)}{1}}{\frac{SSE(x_1,x_2)}{n-p-1}}$$
我们可以利用这个统计量作为一个准则，该准则能确定模型中$x_2$的存在是否会引起误差平方和有一个显著的减少  
通常应用的拒绝法则是：如果p-值$\le\alpha$，则拒绝$H_0$  
## 16.4.1 逐步回归
逐步回归方法的每一步都是从确定已经在模型中的自变量是否应该有被删除的开始。为此，首先对已经在模型中的每一个自变量计算F统计量和对应的p-值   
为了确定一个自变量是否应该从模型中被删除的显著性水平$\alpha$，在Minitab输出中被表示为'Alpha to remove'  
如果有自变量的p-值大于'Alpha to remove'，则具有最大P-值的自变量应该从模型中被删除，并且逐步回归方法开始新的一步 5  
如果没有自变量能从模型中被删除，那么逐步回归方法将试图使另一个自变量进入模型  
为此，首先对没有在模型中的每一个自变量计算F统计量和对应的p-值。为了确定一个自变量是否应该进入模型的显著性水平$\alpha$，在Minitab输出中被表示为'Alpha to enter'  
如果有自变量的p-值小余或等于'Alpha to enter'，则具有最小p-值的自变量将进入模型  
按照这种方式将逐步回归过程继续进行下去，直到没有一个自变量能从模型中被删除，或者没有一个自变量能被增加到模型上为止   
图16-16给出了对Cravens数据应用Minitab逐步回归程序得到的结果，'Alpha to remove'和'Alpha to enter'的值都取0.05 5  

In [ ]:
'''
5
'''
def forward_selected(data, response):
    """前向逐步回归算法，源代码来自https://planspace.org/20150423-forward_selection_with_statsmodels/
    使用Adjusted R-squared来评判新加的参数是否提高回归中的统计显著性
    Linear model designed by forward selection.
    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response
    response: string, name of response column in data
    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
 
    return model

cra_mod_lv2 = forward_selected(cra, 'Sales')
cra_mod_lv2.summary()

In [ ]:
def forward_selected(data, response):
    """前向逐步回归算法，源代码来自https://planspace.org/20150423-forward_selection_with_statsmodels/
    使用 来评判新加的参数是否提高回归中的统计显著性
    Linear model designed by forward selection.
    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response
    response: string, name of response column in data
    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
 
    return model

cra_mod_lv2 = forward_selected(cra, 'Sales')
cra_mod_lv2.summary()